In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [ ]:
texts = ["I love love Programming.", " I love Math.", " I tolerate Biology."]

### Create Bag of Words

#### Count Vectors

Count Vectorizer documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer

class __sklearn.feature_extraction.text.CountVectorizer__(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)[source]¶

CountVectorizer has buildin lowercasing, punctuation & stop words removal 

In [ ]:
cvect = CountVectorizer()
text_cvect = cvect.fit_transform(texts)
cvect.vocabulary_

Results are stored in CSR format, memory effective

In [ ]:
text_cvect

Dense representation:

In [ ]:
text_cvect.todense()

Lets try "binary=True". E.g. create one-hot encoding representation. This is useful for discrete probabilistic models that model binary events rather than integer counts. May be good option for sentiment analysis. 

In [ ]:
cvect = CountVectorizer(binary=True)
text_cvect = cvect.fit_transform(texts)
text_cvect.todense()

Now lets have unigrams &  bigrams

In [ ]:
cvect = CountVectorizer(ngram_range=(1, 2))
text_cvect = cvect.fit_transform(texts)
cvect.vocabulary_

In [ ]:
text_cvect.todense()

Sometimes models can be created based on chars. It allows to capture the most common words parts. No need for stemming/lemmatizaion. 
We can setup max & min number of tokens occurencies with max_df, min_df, max_features

In [ ]:
#cvect = CountVectorizer(ngram_range=(3, 4), analyzer="char", max_features=10)
cvect = CountVectorizer(ngram_range=(3, 4), analyzer="char")
text_cvect = cvect.fit(texts)
cvect.vocabulary_

#### TFIDF vectors

TfidfVectorizer documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer
class __sklearn.feature_extraction.text.TfidfVectorizer__(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None, token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.float64’>, norm=’l2’, use_idf=True, smooth_idf=True, sublinear_tf=False)[source]

In [ ]:
tfidf = TfidfVectorizer(smooth_idf=False, use_idf=False, norm='l1')
text_tfidf = tfidf.fit_transform(texts)
tfidf.vocabulary_

This is a result of term frequency calculation. No idf applied

In [ ]:
text_tfidf.todense()

In the word "love" has lower weight

In [ ]:
tfidf = TfidfVectorizer(smooth_idf=False, use_idf=True, norm='l1')
text_tfidf = tfidf.fit_transform(texts)
text_tfidf.todense()

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 2))
text_tfidf = tfidf.fit_transform(texts)
text_tfidf.todense()

#### Hashing trick

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer

__sklearn.feature_extraction.text.HashingVectorizer__(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, n_features=1048576, binary=False, norm=’l2’, alternate_sign=True, dtype=<class ‘numpy.float64’>)

This text vectorizer implementation uses the hashing trick to find the token string name to feature integer index mapping.

This strategy has several advantages:

 * it is very low memory scalable to large datasets as there is no need to store a vocabulary dictionary in memory
 * it is fast to pickle and un-pickle as it holds no state besides the constructor parameters
 * it can be used in a streaming (partial fit) or parallel pipeline as there is no state computed during fit.
 
There are also a couple of cons (vs using a CountVectorizer with an in-memory vocabulary):

 * there is no way to compute the inverse transform (from feature indices to string feature names) which can be a problem when  trying to introspect which features are most important to a model.
 * there can be collisions: distinct tokens can be mapped to the same feature index. However in practice this is rarely an issue if n_features is large enough (e.g. 2 ** 18 for text classification problems).
 * no IDF weighting as this would render the transformer stateful.

In [ ]:
hashv = HashingVectorizer(n_features=20)
text_hashv = hashv.fit_transform(texts)
text_hashv.todense()

## Co-occurencies matrix

In [ ]:
from collections import defaultdict

In [ ]:
# look at the right implementation
def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        text = [w.replace(".", "") for w in text]
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1
        # formulate the dictionary into dataframe
    vocab = sorted(vocab) 
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [ ]:
co_occurrence(texts, 1)